# Genetic algorithm model calibration

This notebook applies a genetic algorithm to calibrate wall decay coefficients using data from each water quality sensing period.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors
default_colors = plotly.colors.qualitative.Plotly
from bayesian_wq_calibration.simulation import model_simulation, sensor_model_id
from bayesian_wq_calibration.ga import fitness
from bayesian_wq_calibration.plotting import plot_network
from bayesian_wq_calibration.constants import TIMESERIES_DIR

Load sensing data for selected sensing period.

In [2]:
data_period = 18 # 19 calibration events (as at 30 September 2024)
try:
    flow_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-flow.csv")
    pressure_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-pressure.csv")
    wq_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-wq.csv", low_memory=False)
    cl_df = wq_df[wq_df['data_type'] == 'chlorine']

    print(f"Data period: {data_period}")
    print(f"Start datetime: {flow_df['datetime'].unique()[0]}")
    print(f"End datetime: {flow_df['datetime'].unique()[-1]}")
except:
    print(f"Data period {data_period} does not exist.")

Data period: 18
Start datetime: 2024-08-20 00:00:00
End datetime: 2024-08-27 23:45:00


Initialize wall decay coefficients.

In [3]:
wall_grouping = 'single'
wall_coeff_0 = -0.05
wall_coeffs = None

if wall_grouping == 'single':
    wall_coeffs = [wall_coeff_0] * 1
elif wall_grouping == 'diameter-based':
    wall_coeffs = [wall_coeff_0] * 4
elif wall_grouping == 'roughness-based':
    wall_coeffs = [wall_coeff_0] * 8
elif wall_grouping == 'material-based':
    wall_coeffs = [wall_coeff_0] * 3

Compute initial fitness function value.

In [4]:
obj_function = 'mse' # 'mse', 'rmse', 'mae', 'mape'
demand_resolution = 'dma' # 'dma', 'wwmd'

obj_0 = fitness(flow_df, pressure_df, cl_df, wall_coeffs, obj_function=obj_function, wall_grouping=wall_grouping, demand_resolution=demand_resolution)

Error setting PRV settings @ link_2820. Default values used.


In [5]:
obj_0

0.013658945786916846

In [8]:
4 * 24 + 1

97

In [9]:
cl_df

,datetime,bwfl_id,data_type,dma_id,wwmd_id,min,mean,max
4,2024-08-20 00:00:00,BW2,chlorine,2005_2296,"2304, 2301",0.341,0.344,0.347
13,2024-08-20 00:00:00,BW12,chlorine,2296,2297,0.423,0.424,0.425
14,2024-08-20 00:00:00,BW7,chlorine,2296,2294,0.022,0.026,0.029
23,2024-08-20 00:00:00,BW1,chlorine,2296,2297,0.419,0.422,0.424
24,2024-08-20 00:00:00,BW3,chlorine,2296,2293,0.370,0.371,0.372
...,...,...,...,...,...,...,...,...
34539,2024-08-27 23:45:00,BW3,chlorine,2296,2293,0.504,0.505,0.505
34544,2024-08-27 23:45:00,BW4,chlorine,2005,"2301, 2332",0.582,0.582,0.583
34549,2024-08-27 23:45:00,BW5,chlorine,2005_2296,"2293, 2302",0.549,0.549,0.549
34554,2024-08-27 23:45:00,BW6,chlorine,2296,2304,0.298,0.299,0.301
